# Transformer계열 모델에서 사용할 total data

In [1]:
import pandas as pd
import os
import random
import numpy as np

import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

## 1. 데이터 로딩

In [2]:
data_dir = '../data/' # 경로는 상황에 맞춰서 수정해주세요!

# LOAD TRAINDATA
csv_file_path = os.path.join(data_dir, 'train_data.csv') # 데이터는 대회홈페이지에서 받아주세요 :)
train_data = pd.read_csv(csv_file_path)

# LOAD TESTDATA
test_csv_file_path = os.path.join(data_dir, 'test_data.csv')
test_data = pd.read_csv(test_csv_file_path)
test_data['answerCode'] = test_data['answerCode'].replace(-1, np.nan)

# # LEAVE LAST INTERACTION ONLY
# train_df = test_df[test_df['userID'] == test_df['userID'].shift(-1)]
# test_df = test_df[test_df['userID'] != test_df['userID'].shift(-1)]

# df = pd.concat([df, train_df], ignore_index=True)

## 2. Feature Engineering

In [3]:
train_data['train'] = 1
test_data['train'] = 0
total = pd.concat((train_data, test_data), axis=0)
total = total.sort_values('userID')
total.to_csv(data_dir + 'total_data.csv', index=False)

In [4]:
def feature_engineering(data_dir):
    total = pd.read_csv(os.path.join(data_dir, 'total_data.csv'), parse_dates=["Timestamp"])
    
    ## 유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    print('Timestamp')
    total['Timestamp'] = pd.to_datetime(total['Timestamp'])
    total.sort_values(by=['userID','Timestamp'], inplace=True)#.reset_index(drop=True)
    
    ############################## 1. Timestamp ##############################
    # 1-1. hour, month
    print('hour', 'month', 'dayofweek')
    total['hour'] = total['Timestamp'].dt.hour
    total['month'] = total['Timestamp'].dt.month
    total['dayofweek'] = total['Timestamp'].dt.dayofweek
    
    # 1-2. 기존 elapsed
    print('elapsed')    # 직전 문제 풀이 소요 시간
    diff = total.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
    diff = diff.fillna(pd.Timedelta(seconds=0))
    diff = diff['Timestamp'].apply(lambda x: x.total_seconds())
    total['elapsed'] = diff
    total['elapsed'] = total['elapsed'].apply(lambda x: x if x < 600 and x>=0 else 0)

#     print('current elapsed')    # 현재 문제 풀이 소요 시간
#     diff = total.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().shift(-1).fillna(method='ffill') # 마지막 문제 소요시간을 직전 문제 시간으로 채움
#     diff = diff['Timestamp'].apply(lambda x: x.total_seconds())
#     total['current_elapsed'] = diff
#     total['current_elapsed'] = total['current_elapsed'].apply(lambda x: x if x < 600 and x>=0 else np.nan)
#     total['current_elapsed'] = total['current_elapsed'].fillna(method='ffill')

    print('time class')
    total['timeClass'] = total['elapsed']//5 + 1
    
#     print('day_diff')
#     day_diff = total.loc[:,['userID','Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
#     day_diff = day_diff['Timestamp'].apply(lambda x: x.days)
#     total['day_diff'] = day_diff
#     total['day_diff'] = total['day_diff'].apply(lambda x: x if x < 1 and x>=0 else 1)   # 1일 이상 간격 1일로 클립
 
    # 1-3. solved_time
    # 같은 문제 몇번째 푸는지
    print('same_item_cnt')
    total['same_item_cnt'] = total.groupby(['userID', 'assessmentItemID']).cumcount() + 1
   
    # 유저, test, same_item_cnt 구분했을 때 **이전** 문제 푸는데 걸린 시간 > shift, fillna
    print('solved_time_prior')
    diff_shift = total.loc[:, ['userID', 'testId', 'Timestamp', 'same_item_cnt']].groupby(['userID', 'testId', 'same_item_cnt']).diff()#.shift(-1)
    diff_shift = diff_shift['Timestamp'].apply(lambda x: x.total_seconds())
    total['solved_time_prior'] = diff_shift
    total['solved_time_prior'].fillna(0, inplace=True)
    total['solved_time_prior'] = total['solved_time_prior'].apply(lambda x: x if x < 600 and x>=0 else 0)
    # total['solved_time_prior'] = total.groupby(['userID', 'testId', 'same_item_cnt'])['solved_time_prior'].apply(lambda x:x.fillna(x.mean()))
    
    # 유저, test, same_item_cnt 구분했을 때 **이전** 문제 푸는데 걸린 시간 > shift, fillna
    print('solved_time')
    diff_shift = total.loc[:, ['userID', 'testId', 'Timestamp', 'same_item_cnt']].groupby(['userID', 'testId', 'same_item_cnt']).diff().shift(-1)
    diff_shift = diff_shift['Timestamp'].apply(lambda x: x.total_seconds())
    total['solved_time'] = diff_shift
    total['solved_time'] = total['solved_time'].apply(lambda x: x if x < 600 and x>=0 else 0)
    total['solved_time'] = total.groupby(['userID', 'testId', 'same_item_cnt'])['solved_time'].apply(lambda x:x.fillna(x.mean()))
    
    
    ############################## 2. agg값 ##############################
    print('bigcat, smallcat')
    total['bigcat'] = total['testId'].str[2]
    total['smallcat'] = total['testId'].str[7:10]
    
    # # Bigcat_class
    # print('Bigcat_class')
    # total['Bigcat_class'] = round(total['Bigcat_avg']*10)
    
    # # 맞은 사람의 문제별 평균 풀이시간
    # total = total.set_index('assessmentItemID')
    # total['Item_mean_solved_time'] = total[total['answerCode'] == 1].groupby('assessmentItemID')['solved_time_shift'].mean()
    # total = total.reset_index(drop = False)

    ## 2-1. 유저/문제/시험지/카테고리/태그별 평균 정답률
    print('avg')
    total['user_avg'] = total.groupby('userID')['answerCode'].transform('mean')
    total['item_avg'] = total.groupby('assessmentItemID')['answerCode'].transform('mean')
    total['test_avg'] = total.groupby('testId')['answerCode'].transform('mean')
    total['bigcat_avg'] = total.groupby('bigcat')['answerCode'].transform('mean')
    total['tag_avg'] = total.groupby('KnowledgeTag')['answerCode'].transform('mean')

    ## 2-2. 유저/문제/시험지/카테고리/태그별 평균 풀이시간
    print('time_avg')
    total['user_time_avg'] = total.groupby('userID')['solved_time'].transform('mean')
    total['item_time_avg'] = total.groupby('assessmentItemID')['solved_time'].transform('mean')
    total['test_time_avg'] = total.groupby('testId')['solved_time'].transform('mean')
    total['bigcat_time_avg'] = total.groupby('bigcat')['solved_time'].transform('mean')
    total['tag_time_avg'] = total.groupby('KnowledgeTag')['solved_time'].transform('mean')
     
    ## 2-3. 유저/문제/시험지/카테고리/태그별 정답률 표준편차
    print('std')
    total['user_std'] = total.groupby('userID')['answerCode'].transform('std')
    total['item_std'] = total.groupby('assessmentItemID')['answerCode'].transform('std')
    total['test_std'] = total.groupby('testId')['answerCode'].transform('std')
    total['bigcat_std'] = total.groupby('bigcat')['answerCode'].transform('std')
    total['tag_std'] = total.groupby('KnowledgeTag')['answerCode'].transform('std')
    
    ## 2-5. 유저/문제/시험지/카테고리/태그별 누적 개수
    print('cum_cnt')
    total['user_cum_cnt'] = total.groupby('userID')['answerCode'].cumcount() + 1
    total['item_cum_cnt'] = total.groupby('assessmentItemID')['answerCode'].cumcount() + 1
    total['test_cum_cnt'] = total.groupby('testId')['answerCode'].cumcount() + 1
    total['bigcat_cum_cnt'] = total.groupby('bigcat')['answerCode'].cumcount() + 1
    total['tag_cum_cnt'] = total.groupby('KnowledgeTag')['answerCode'].cumcount() + 1
    
    ## 2-6. 유저/문제/시험지/카테고리/태그별 누적정답률
    print('cum_acc')
    total['user_cor_cum_cnt'] = total.groupby('userID')['answerCode'].cumsum()
    total['item_cor_cum_cnt'] = total.groupby('assessmentItemID')['answerCode'].cumsum()
    total['test_cor_cum_cnt'] = total.groupby('testId')['answerCode'].cumsum()
    total['bigcat_cor_cum_cnt'] = total.groupby('bigcat')['answerCode'].cumsum()
    total['tag_cor_cum_cnt'] = total.groupby('KnowledgeTag')['answerCode'].cumsum()
    
    total['user_cum_acc'] = total['user_cor_cum_cnt']/total['user_cum_cnt']
    total['item_cum_acc'] = total['item_cor_cum_cnt']/total['item_cum_cnt']
    total['test_cum_acc'] = total['test_cor_cum_cnt']/total['test_cum_cnt']
    total['bigcat_cum_acc'] = total['bigcat_cor_cum_cnt']/total['bigcat_cum_cnt']
    total['tag_cum_acc'] = total['tag_cor_cum_cnt']/total['tag_cum_cnt']
    
    ## 2-7. 유저의 시험지/카테고리/태그별 누적 정답률
    print('cum_acc per user')
    total['test_cum_cnt_per_user'] = total.groupby(['userID', 'testId'])['answerCode'].cumcount() + 1
    total['bigcat_cum_cnt_per_user'] = total.groupby(['userID', 'bigcat'])['answerCode'].cumcount() + 1
    total['tag_cum_cnt_per_user'] = total.groupby(['userID', 'KnowledgeTag'])['answerCode'].cumcount() + 1
    
    total['test_cor_cum_per_user'] = total.groupby(['userID', 'testId'])['answerCode'].cumsum()
    total['bigcat_cor_cum_per_user'] = total.groupby(['userID', 'bigcat'])['answerCode'].cumsum()
    total['tag_cor_cum_per_user'] = total.groupby(['userID', 'KnowledgeTag'])['answerCode'].cumsum()
    
    total['test_cum_acc_per_user'] = total['test_cor_cum_per_user']/total['test_cum_cnt_per_user']
    total['bigcat_cum_acc_per_user'] = total['bigcat_cor_cum_per_user']/total['bigcat_cum_cnt_per_user']
    total['tag_cum_acc_per_user'] = total['tag_cor_cum_per_user']/total['tag_cum_cnt_per_user']
    
    
    ############################## 3. current ##############################
    ## 3-1. 유저의 해당 문제지 평균 정답률/풀이시간
    print('current')
    total['user_cur_avg'] = total.groupby(['userID', 'testId', 'same_item_cnt'])['answerCode'].transform('mean')
    total['user_cur_time_avg'] = total.groupby(['userID', 'testId', 'same_item_cnt'])['solved_time'].transform('mean')
    
    print('user_rolling')
    total['user_rec_avg_rolling5'] = total.groupby('userID')['answerCode'].transform(lambda x:x.rolling(5, min_periods=1).mean())
    total['user_rec_time_avg_rolling5'] = total.groupby('userID')['solved_time'].transform(lambda x:x.rolling(5, min_periods=1).mean())

    total['user_rec_avg_rolling7'] = total.groupby('userID')['answerCode'].transform(lambda x:x.rolling(7, min_periods=1).mean())
    total['user_rec_time_avg_rolling7'] = total.groupby('userID')['solved_time'].transform(lambda x:x.rolling(7, min_periods=1).mean())
    
    total['user_rec_avg_rolling10'] = total.groupby('userID')['answerCode'].transform(lambda x:x.rolling(10, min_periods=1).mean())
    total['user_rec_time_avg_rolling10'] = total.groupby('userID')['solved_time'].transform(lambda x:x.rolling(10, min_periods=1).mean())
    
    print('item_rolling')
    total['item_rec_avg_rolling5'] = total.groupby('assessmentItemID')['answerCode'].transform(lambda x:x.rolling(5, min_periods=1).mean())
    total['item_rec_time_avg_rolling5'] = total.groupby('assessmentItemID')['solved_time'].transform(lambda x:x.rolling(5, min_periods=1).mean())

    total['item_rec_avg_rolling7'] = total.groupby('assessmentItemID')['answerCode'].transform(lambda x:x.rolling(7, min_periods=1).mean())
    total['item_rec_time_avg_rolling7'] = total.groupby('assessmentItemID')['solved_time'].transform(lambda x:x.rolling(7, min_periods=1).mean())
    
    total['item_rec_avg_rolling10'] = total.groupby('assessmentItemID')['answerCode'].transform(lambda x:x.rolling(10, min_periods=1).mean())
    total['item_rec_time_avg_rolling10'] = total.groupby('assessmentItemID')['solved_time'].transform(lambda x:x.rolling(10, min_periods=1).mean())
    
    
#     # 2-1 유저/문제별 최근 정답횟수 
#     print('retCount_correct_answer')
#     total['user_retCount_correct_answer'] = total.groupby('userID')['answerCode'].transform(lambda x:x.rolling(5, min_periods=1).sum().shift(1))
#     total['item_retCount_correct_answer'] = total.groupby('assessmentItemID')['answerCode'].transform(lambda x:x.rolling(5, min_periods=1).sum().shift(1))
    

#     # 2-1 유저/문제별 최근 정답률
#     print('retCumacc')
#     total['user_retCount'] = total.groupby('userID')['answerCode'].transform(lambda x:x.rolling(5, min_periods=1).count().shift(1))
#     total['user_retCumacc'] = total['user_retCount_correct_answer']/total['user_retCount']

#     total['item_retCount'] = total.groupby('assessmentItemID')['answerCode'].transform(lambda x:x.rolling(5, min_periods=1).count().shift(1))
#     total['item_retCumacc'] = total['item_retCount_correct_answer']/total['item_retCount']
    
    
    ############################## 4. etc ##############################
    ## 문제 번호 추가
    print('item_num')
    total['item_num'] = total['assessmentItemID'].str[7:]
    
    ## 문제 푼 순서 추가 > 상대적 순서?
    print('item_seq')
    total['item_seq'] = total.groupby(['userID', 'testId', 'same_item_cnt']).cumcount()+1
    
    
    
    # total[['user_correct_answer','item_correct_answer',
    #        'user_Cumacc',
    #        'user_Bigcat_correct_answer', 'user_Bigcat_Cumacc', 
    #        'user_retCount_correct_answer', 'item_retCount_correct_answer',
    #        'user_retCount', 'user_retCumacc',
    #        'item_retCount', 'item_retCumacc']] = total[['user_correct_answer', 'item_correct_answer',
    #                                                     'user_Cumacc',
    #                                                     'user_Bigcat_correct_answer', 'user_Bigcat_Cumacc', 
    #                                                     'user_retCount_correct_answer', 'item_retCount_correct_answer',
    #                                                     'user_retCount', 'user_retCumacc',
    #                                                     'item_retCount', 'item_retCumacc']].fillna(0)
    
    
    
    
    return total

In [5]:
total = feature_engineering(data_dir)
total.head(15)

Timestamp
hour month dayofweek
elapsed
time class
same_item_cnt
solved_time_prior
solved_time


/tmp/ipykernel_22409/1888887427.py:60: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  total['solved_time'] = total.groupby(['userID', 'testId', 'same_item_cnt'])['solved_time'].apply(lambda x:x.fillna(x.mean()))


bigcat, smallcat
avg
time_avg
std
cum_cnt
cum_acc
cum_acc per user
current
user_rolling
item_rolling
item_num
item_seq


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,train,hour,month,dayofweek,...,user_rec_avg_rolling10,user_rec_time_avg_rolling10,item_rec_avg_rolling5,item_rec_time_avg_rolling5,item_rec_avg_rolling7,item_rec_time_avg_rolling7,item_rec_avg_rolling10,item_rec_time_avg_rolling10,item_num,item_seq
0,0,A060001001,A060000001,1.0,2020-03-24 00:17:11,7224,1,0,3,1,...,1.000000,3.000000,1.0,3.0,1.0,3.0,1.0,3.0,001,1
491,0,A060001002,A060000001,1.0,2020-03-24 00:17:14,7225,1,0,3,1,...,1.000000,5.500000,1.0,8.0,1.0,8.0,1.0,8.0,002,2
492,0,A060001003,A060000001,1.0,2020-03-24 00:17:22,7225,1,0,3,1,...,1.000000,6.000000,1.0,7.0,1.0,7.0,1.0,7.0,003,3
493,0,A060001004,A060000001,1.0,2020-03-24 00:17:29,7225,1,0,3,1,...,1.000000,6.250000,1.0,7.0,1.0,7.0,1.0,7.0,004,4
494,0,A060001005,A060000001,1.0,2020-03-24 00:17:36,7225,1,0,3,1,...,1.000000,7.200000,1.0,11.0,1.0,11.0,1.0,11.0,005,5
495,0,A060001007,A060000001,1.0,2020-03-24 00:17:47,7225,1,0,3,1,...,1.000000,6.000000,1.0,0.0,1.0,0.0,1.0,0.0,007,6
496,0,A060003001,A060000003,0.0,2020-03-26 05:52:03,7226,1,5,3,3,...,0.857143,6.142857,0.0,7.0,0.0,7.0,0.0,7.0,001,1
497,0,A060003002,A060000003,1.0,2020-03-26 05:52:10,7226,1,5,3,3,...,0.875000,13.375000,1.0,64.0,1.0,64.0,1.0,64.0,002,2
498,0,A060003003,A060000003,1.0,2020-03-26 05:53:14,7226,1,5,3,3,...,0.888889,13.555556,1.0,15.0,1.0,15.0,1.0,15.0,003,3
500,0,A060003004,A060000003,1.0,2020-03-26 05:53:29,7226,1,5,3,3,...,0.900000,14.100000,1.0,19.0,1.0,19.0,1.0,19.0,004,4


In [10]:
fill_columns = ['user_cor_cum_cnt', 'item_cor_cum_cnt', 'test_cor_cum_cnt', 'bigcat_cor_cum_cnt', 'tag_cor_cum_cnt',
               'user_cum_acc', 'item_cum_acc', 'test_cum_acc', 'bigcat_cum_acc', 'tag_cum_acc',
               'test_cor_cum_per_user', 'bigcat_cor_cum_per_user', 'tag_cor_cum_per_user',
               'test_cum_acc_per_user', 'bigcat_cum_acc_per_user', 'tag_cum_acc_per_user',
               'item_rec_avg_rolling5', 'item_rec_avg_rolling7', 'item_rec_avg_rolling10'
               ]

total[fill_columns] = total[fill_columns].fillna(method='ffill')

In [11]:
total.isna().sum().sum()

744

In [12]:
# save total_data
total.to_csv(data_dir + 'total_data_hyein5.csv', index=False)